In [1]:
from keras.preprocessing.image import load_img, img_to_array
from keras import backend as K
import numpy as np
from keras.applications import vgg19
from scipy.optimize import fmin_l_bfgs_b
import matplotlib.pyplot 
import time

In [2]:
img_height = 512
img_width = 512

In [3]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [4]:
def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [5]:
target_image = K.constant(preprocess_image("../../appli/aai.jpg"))
style_reference_image = K.constant(preprocess_image("../../appli/index.jpg"))
combination_image = K.placeholder((1, img_height, img_width, 3))

In [6]:
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)

In [7]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(3, 512, 512, 3)]        0         
_________________________________________________________________
block1_conv1 (Conv2D)        (3, 512, 512, 64)         1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (3, 512, 512, 64)         36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (3, 256, 256, 64)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (3, 256, 256, 128)        73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (3, 256, 256, 128)        147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (3, 128, 128, 128)        0     

In [8]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [9]:
def total_variation_loss(x):
    a = K.square(
            x[:, :img_height - 1, :img_width - 1, :] -
            x[:, 1:, :img_width - 1, :])
    b = K.square(
            x[:, :img_height - 1, :img_width - 1, :] -
            x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [10]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

total_variation_weight = 1e-5
style_weight = 1.
content_weight = 0.0003125

In [11]:
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

loss += content_weight * content_loss(target_image_features,
                                      combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]

    reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]

    sl = style_loss(reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl

loss += total_variation_weight * total_variation_loss(combination_image)

RuntimeError: Variable += value not supported. Use variable.assign_add(value) to modify the variable value and variable = variable + value to get a new Tensor object.

In [ ]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()
grads = K.gradients(loss, combination_image)[0]
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

In [ ]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])

        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [ ]:
evaluator = Evaluator()

In [ ]:
result_prefix = 'my_result'
iterations = 300
x = preprocess_image("/content/paysage.jpg")
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()


    x, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                     x,
                                     fprime=evaluator.grads,
                                     maxfun=20)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))  
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    matplotlib.pyplot.imsave(fname, img)
    print('Image saved as', fname)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))